# imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle 
import time 
import datetime
import warnings
from autogluon.tabular import TabularDataset, TabularPredictor

warnings.filterwarnings('ignore')

In [2]:
df_train1 = pd.read_csv('~/Dropbox/Data/df_train1.csv')
df_train2 = pd.read_csv('~/Dropbox/Data/df_train2.csv')
df_train3 = pd.read_csv('~/Dropbox/Data/df_train3.csv')
df_train4 = pd.read_csv('~/Dropbox/Data/df_train4.csv')
df_train5 = pd.read_csv('~/Dropbox/Data/df_train5.csv')
df_train6 = pd.read_csv('~/Dropbox/Data/df_train6.csv')
df_train7 = pd.read_csv('~/Dropbox/Data/df_train7.csv')
df_train8 = pd.read_csv('~/Dropbox/Data/df_train8.csv')
df_test = pd.read_csv('~/Dropbox/Data/df_test.csv')

In [3]:
(df_train1.shape, df_train1.is_fraud.mean()), (df_test.shape, df_test.is_fraud.mean())

(((734003, 22), 0.005728859418830713), ((314572, 22), 0.005725239372862174))

In [4]:
_df1 = pd.concat([df_train1, df_test])
_df2 = pd.concat([df_train2, df_test])
_df3 = pd.concat([df_train3, df_test])
_df4 = pd.concat([df_train4, df_test])
_df5 = pd.concat([df_train5, df_test])
_df6 = pd.concat([df_train6, df_test])
_df7 = pd.concat([df_train7, df_test])
_df8 = pd.concat([df_train8, df_test])

In [5]:
_df1_mean = _df1.is_fraud.mean()
_df2_mean = _df2.is_fraud.mean()
_df3_mean = _df3.is_fraud.mean()
_df4_mean = _df4.is_fraud.mean()
_df5_mean = _df5.is_fraud.mean()
_df6_mean = _df6.is_fraud.mean()
_df7_mean = _df7.is_fraud.mean()
_df8_mean = _df8.is_fraud.mean()

In [6]:
def auto_amt_ver0503(df_tr, df_tst, _df_mean):
    df_tr = df_tr[["amt","is_fraud"]]
    df_tst = df_tst[["amt","is_fraud"]]     
    tr = TabularDataset(df_tr)
    tst = TabularDataset(df_tst)
    predictr = TabularPredictor(label="is_fraud", verbosity=0)
    t1 = time.time()
    predictr.fit(tr)
    t2 = time.time()
    time_diff = t2 - t1
    models = predictr._trainer.model_graph.nodes
    results = []
    for model_name in models:
    # 모델 평가
        eval_result = predictr.evaluate(tst, model=model_name)

    # 결과를 데이터프레임에 추가
        results.append({'model': model_name, 
                        'acc': eval_result['accuracy'], 
                        'pre': eval_result['precision'], 
                        'rec': eval_result['recall'], 
                        'f1': eval_result['f1'], 
                        'auc': eval_result['roc_auc']})
        
    model = []
    time_diff = []
    acc = []
    pre = []
    rec = []
    f1 = [] 
    auc = [] 
    graph_based = []
    method = [] 
    throw_rate = [] 
    train_size = []
    train_cols = []
    train_frate = []
    test_size = []
    test_frate = []
    hyper_params = [] 
    
    for result in results:
        model_name = result['model']
        model.append(model_name)
        time_diff.append(None)  # 각 모델별로 학습한 시간을 나타내고 싶은데 잘 안됨
        acc.append(result['acc']) 
        pre.append(result['pre'])
        rec.append(result['rec'])
        f1.append(result['f1'])
        auc.append(result['auc'])
        graph_based.append(False) 
        method.append('Autogluon') 
        throw_rate.append(_df_mean)
        train_size.append(len(tr))
        train_cols.append([col for col in tr.columns if col != 'is_fraud'])
        train_frate.append(tr.is_fraud.mean())
        test_size.append(len(tst))
        test_frate.append(tst.is_fraud.mean())
        hyper_params.append(None)
        
    df_results = pd.DataFrame(dict(
        model=model,
        time=time_diff,
        acc=acc,
        pre=pre,
        rec=rec,
        f1=f1,
        auc=auc,
        graph_based=graph_based,
        method=method,
        throw_rate=throw_rate,  
        train_size=train_size,
        train_cols=train_cols,
        train_frate=train_frate,
        test_size=test_size,
        test_frate=test_frate,
        hyper_params=hyper_params
    ))    
    ymdhms = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d-%H%M%S') 
    df_results.to_csv(f'../results2/{ymdhms}-Autogluon.csv',index=False)
    return df_results

In [15]:
auto_amt_ver0503(df_train1, df_test, _df1_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.994218,0.489865,0.241532,0.323540,0.782174,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
1,KNeighborsDist,None,0.993582,0.403197,0.252082,0.310215,0.750937,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
2,LightGBMXT,None,0.994275,0.000000,0.000000,0.000000,0.765047,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
3,LightGBM,None,0.994275,0.000000,0.000000,0.000000,0.957828,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
4,RandomForestGini,None,0.993140,0.363844,0.264853,0.306555,0.816185,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
5,RandomForestEntr,None,0.993140,0.363844,0.264853,0.306555,0.816185,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
6,CatBoost,None,0.994275,0.000000,0.000000,0.000000,0.870449,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
7,ExtraTreesGini,None,0.993483,0.393316,0.254858,0.309299,0.840480,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
8,ExtraTreesEntr,None,0.993493,0.393966,0.253748,0.308680,0.839967,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
9,NeuralNetFastAI,None,0.994275,0.000000,0.000000,0.000000,0.943377,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None


In [16]:
auto_amt_ver0503(df_train2, df_test, _df2_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.993753,0.440837,0.339256,0.383433,0.816442,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
1,KNeighborsDist,None,0.992984,0.378443,0.350916,0.364160,0.793232,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
2,LightGBMXT,None,0.994275,0.000000,0.000000,0.000000,0.765797,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
3,LightGBM,None,0.994275,0.000000,0.000000,0.000000,0.957293,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
4,RandomForestGini,None,0.992393,0.341880,0.355358,0.348489,0.840338,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
5,RandomForestEntr,None,0.992393,0.341880,0.355358,0.348489,0.840338,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
6,CatBoost,None,0.994253,0.496150,0.250416,0.332841,0.936583,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
7,ExtraTreesGini,None,0.992866,0.371296,0.354803,0.362862,0.860650,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
8,ExtraTreesEntr,None,0.992911,0.373898,0.353137,0.363221,0.858746,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
9,NeuralNetFastAI,None,0.993804,0.435315,0.276513,0.338200,0.929382,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None


In [17]:
auto_amt_ver0503(df_train3, df_test, _df3_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.988111,0.269002,0.626874,0.376459,0.881531,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
1,KNeighborsDist,None,0.986839,0.239242,0.595780,0.341394,0.866643,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
2,LightGBMXT,None,0.994275,0.000000,0.000000,0.000000,0.781026,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
3,LightGBM,None,0.990857,0.327004,0.564131,0.414018,0.962248,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
4,RandomForestGini,None,0.985933,0.221444,0.579123,0.320381,0.892199,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
5,RandomForestEntr,None,0.985933,0.221444,0.579123,0.320381,0.892199,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
6,CatBoost,None,0.989109,0.289889,0.622432,0.395554,0.947419,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
7,ExtraTreesGini,None,0.986935,0.240853,0.595780,0.343031,0.901395,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
8,ExtraTreesEntr,None,0.986881,0.239937,0.595780,0.342101,0.900649,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
9,NeuralNetFastAI,None,0.992202,0.358630,0.459189,0.402727,0.892828,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None


In [ ]:
auto_amt_ver0503(df_train4, df_test, _df4_mean)

In [ ]:
auto_amt_ver0503(df_train5, df_test, _df5_mean)

In [ ]:
auto_amt_ver0503(df_train6, df_test, _df6_mean)

In [ ]:
auto_amt_ver0503(df_train7, df_test, _df7_mean)

In [ ]:
auto_amt_ver0503(df_train8, df_test, _df8_mean)